# Import libraries

In [1]:
# Import libgraph library containing all functions created
import libgraph
from importlib import reload
reload(libgraph)

# Import external libraries to compute the dataset
import networkx as nx
import pandas as pd
import statistics
import collections
import timeit

# Printing 
class color:
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

# Reading data

In this section we are reading every file for the following sections

### Reading reduced edges file

In [2]:
# Variable containing 'wiki-topcats-reduced.txt'
reduced = pd.read_csv('wiki-topcats-reduced.txt', names=['source', 'destination'], sep='\t')

In [3]:
# See the head of the file
reduced.head()

,source,destination
0,52,401135
1,52,1069112
2,52,1163551
3,62,12162
4,62,167659


In [4]:
# Number of edges
print('Number of edges:', len(reduced)) 

Number of edges: 2645247


In [5]:
# Set unique total nodes in reduced
set_nodes = set([*list(reduced['source']), *list(reduced['destination'])]) 

In [6]:
# Number of nodes
print('Number of nodes:', len(set_nodes)) 

Number of nodes: 461193


### Reading page names file

In [7]:
# Variable containing 'wiki-topcats-page-names.txt'
names_list = libgraph.read_names_list('wiki-topcats-page-names.txt')

In [8]:
# Print some results of the readed file
print(*names_list[0:5], sep="; ")

Chiasmal syndrome; Kleroterion; Pinakion; LyndonHochschildSerre spectral sequence; Zariski's main theorem


In [9]:
# See number of names in the list
print(len(names_list))

1791489


### Reading categories file

In this step, we are reading categories dictionary keeping only categories with more than 3500 nodes and with nodes contained in the reduced edges file

In [10]:
# Variable containing 'wiki-topcats-categories.txt'
categories = libgraph.read_categories_list('wiki-topcats-categories.txt')

In [11]:
# Variable containing the length of each category
length_categories = {key: len(value) for key, value in categories.items()}
# Print categories keys and length of the contained list
print(*length_categories.items())
# Delete variable because is not used anymore
del length_categories

('English_footballers', 9237) ('The_Football_League_players', 9467) ('Association_football_forwards', 6959) ('Association_football_goalkeepers', 3997) ('Association_football_midfielders', 8270) ('Association_football_defenders', 6668) ('Living_people', 418223) ('Year_of_birth_unknown', 3760) ('Harvard_University_alumni', 6154) ('Major_League_Baseball_pitchers', 6580) ('Members_of_the_United_Kingdom_Parliament_for_English_constituencies', 6546) ('Indian_films', 5913) ('Year_of_death_missing', 7851) ('English_cricketers', 3813) ('Year_of_birth_missing_(living_people)', 34721) ('Rivers_of_Romania', 7729) ('Main_Belt_asteroids', 13704) ('Asteroids_named_for_people', 5701) ('English-language_albums', 4853) ('English_television_actors', 3501) ('British_films', 4551) ('English-language_films', 22699) ('American_films', 15302) ('Fellows_of_the_Royal_Society', 3697) ('People_from_New_York_City', 4888) ('American_Jews', 3542) ('American_television_actors', 11661) ('American_film_actors', 13938) 

In [12]:
# Print number of categories
print("Number of categories:", len(categories.keys()))

Number of categories: 35


Now we have to intersect the dictionary of categories with the set of nodes in reduced file:

In [13]:
# Intersect categories and set_nodes of reduced
categories_reduced = libgraph.intersect_categories_reduced(categories, set_nodes)

In [14]:
# Variable containing the length of each category
length_categories_reduced = {key: len(value) for key, value in categories_reduced.items()}
# Print categories keys and length of the contained list
print(*length_categories_reduced.items())
# Delete variable because is not used anymore
del length_categories_reduced, categories

('English_footballers', 7538) ('The_Football_League_players', 7814) ('Association_football_forwards', 5097) ('Association_football_goalkeepers', 3737) ('Association_football_midfielders', 5827) ('Association_football_defenders', 4588) ('Living_people', 348300) ('Harvard_University_alumni', 5549) ('Major_League_Baseball_pitchers', 5192) ('Members_of_the_United_Kingdom_Parliament_for_English_constituencies', 6491) ('Indian_films', 5568) ('Year_of_death_missing', 4122) ('Year_of_birth_missing_(living_people)', 28498) ('Rivers_of_Romania', 7729) ('Main_Belt_asteroids', 11660) ('Asteroids_named_for_people', 4895) ('English-language_albums', 4760) ('British_films', 4422) ('English-language_films', 22463) ('American_films', 15159) ('People_from_New_York_City', 4614) ('American_television_actors', 11531) ('American_film_actors', 13865) ('Debut_albums', 7561) ('Black-and-white_films', 10759) ('Year_of_birth_missing', 4346) ('Place_of_birth_missing_(living_people)', 5532) ('American_military_per

In [15]:
print("Number of categories_reduced:", len(categories_reduced.keys()))

Number of categories_reduced: 29


# [RQ1] Build the graph

In this RQ we are going to build the graph G=(V, E), where V is the set of articles and E the hyperlinks among them. For this RQ we are using networkx library.

As we have seen before, in top_cats_reduced we have directional edges, and not every node goes both ways so we are going to build a directed graph.

We are going to create a graph inserting the nodes and then the edges:

In [16]:
# Create a directed Graph
Gdir = nx.DiGraph()
# Insert Nodes
Gdir.add_nodes_from(set_nodes)
# Add Edges
tuples_source_destination = [tuple(x) for x in reduced.values] # List of tuples of source destination
Gdir.add_edges_from(tuples_source_destination)
# Delete variable because is not used anymore
del tuples_source_destination, reduced, set_nodes

Let's see the resulting graph and analize it:
- Print info of the graph
- Compute density

In [17]:
# Print info of the directed graph
print(nx.info(Gdir))

Name: 
Type: DiGraph
Number of nodes: 461193
Number of edges: 2645247
Average in degree:   5.7357
Average out degree:   5.7357


In [18]:
# Print density
print('Density of the graph:',nx.density(Gdir))

Density of the graph: 1.2436602635647606e-05


### Summary graph results:

- It is a **DIRECTED graph** because the edges given are source-destination
- The number of nodes: **461,193**
- The number of edges: **2,645,247**
- The average node degree: **Average in degree:   5.7357 | Average out degree:   5.7357** Each node has on average 5.7 nodes in and 5.7 nodes out
- Is the graph dense?: **density = 1.24e-05** The density is 0 for a graph without edges and 1 for a complete graph. So it is not dense.

# [RQ2] Block Ranking

In this RQ we are going to obtain a block-ranking, where the blocks are represented by the categories. In particular, we want: **block_ranking = [C0, C1, C2...] and their score**. Each category corresponds to a list of nodes.

To order block ranking we are using this algorithm:
**distance(Co, Ci) = median(ShortestPath(Co, Ci))**

## -- 2.0) Test speed and reliability of ShortestPath algorithms [THIS PART IS NOT ASKED] --

First of all we are going to create an algorithm to compute shortest path between edges. We have made 2 algorithms and we are comparing results and speed with networkx implementation. So we are running the same test to compute the shortest_path_length between 2 categories:

- source_category = 'English_footballers' (taking X nodes of this category)
- target_category = 'The_Football_League_players' (taking all nodes of category)

Tests description:

- **Test 1** --> Using shortest path function from networkx
- **Test 2** --> Using shortest path function implemented by me
- **Test 3** --> Using the other shortest path function implemented by me

In [19]:
# Variable containing adjency dictionary of the directed graph
dic_adj = dict(Gdir.adjacency())

### -- Test 1 (using networkX algorithm) --
We are testing **nx.shortest_path_length** algorithm from networkx library

In [20]:
#Number of nodes of C0 taking for running each test
n_C0_nodes = 100

In [21]:
# Function for timing test_1_networkx
def test_1_networkx():
    CxCy = []
    # Input category to make block ranking
    source_category = 'English_footballers'
    target_category = 'The_Football_League_players'
    # List of nodes in each category
    C0 = list(categories_reduced.get(source_category)) #C0
    CI = set(categories_reduced.get(target_category)) #C1
    # Run algorithm
    for s_value in C0[:n_C0_nodes]:
        for t_value in CI:
            try: CxCy.append(nx.shortest_path_length(Gdir, source=s_value, target=t_value))
            except: pass

    print("Number of shortest path computed:", len(CxCy))
    print("Median:", statistics.median(CxCy))
    return statistics.median(CxCy)

# Run speed and reliability test
print('Execution time:', round(timeit.timeit(test_1_networkx, number = 1),2), 's')

Number of shortest path computed: 391744
Median: 5.0
Execution time: 99.64 s


### -- Test 2 (using own implementation algorithm) --
We are testing **libgraph.shortest_path_bfs_list** algorithm developed

In [22]:
# Function for timing test_2_spbl
def test_2_spbl():
    CxCy = []
    # Input category to make block ranking
    source_category = 'English_footballers'
    target_category = 'The_Football_League_players'
    # List of nodes in each category
    C0 = list(categories_reduced.get(source_category)) #C0
    CI = set(categories_reduced.get(target_category)) #C1
    # Run algorithm
    for s_value in C0[:n_C0_nodes]:
        CxCy.extend(libgraph.shortest_path_bfs_list(dic_adj, s_value, CI))

    print("Number of shortest path computed:", len(CxCy))
    print("Median:", statistics.median(CxCy))
    return statistics.median(CxCy)

# Run speed and reliability test
print('Execution time:', round(timeit.timeit(test_2_spbl, number = 1),2), 's')

Number of shortest path computed: 391744
Median: 5.0
Execution time: 72.75 s


### -- Test 3 (using other own implementation algorithm) --
We are testing **libgraph.shortest_path_bfs_list_list** algorithm developed

In [23]:
# Function for timing test_3_spbll
def test_3_spbll():
    CxCy = []
    # Input category to make block ranking
    source_category = 'English_footballers'
    target_category = 'The_Football_League_players'
    # List of nodes in each category
    C0 = list(categories_reduced.get(source_category)) #C0
    CI = set(categories_reduced.get(target_category)) #C1
    # Run algorithm
    CxCy.extend(libgraph.shortest_path_bfs_list_list(dic_adj, set(C0[:n_C0_nodes]), CI))

    print("Number of shortest path computed:", len(CxCy))
    print("Median:", statistics.median(CxCy))
    return statistics.median(CxCy)

# Run speed and reliability test
print('Execution time:', round(timeit.timeit(test_3_spbll, number = 1),2), 's')

Number of shortest path computed: 391744
Median: 5.0
Execution time: 117.93 s


### -- Testing results: --

As we can see we have obtained same results in the three test taking **100 nodes in C0** and **all nodes in CI**. But comparing speed we have that:

- Test 1 (networkx algorithm): 99.64 s
- **Test 2 (shortest_path_bfs_list): 72.75 s**
- Test 3 (shortest_path_bfs_list_list): 117.93 s

So from now on we are using **shortest_path_bfs_list** algorithm developed.

## 2.1) Building Block_Ranking
Now we are going to build block ranking using our own implementation of shortest path algorithm based in test 2. We are going to compare **50 nodes of C0 with all nodes in CI** to build the block ranking. With our algorithm it will take approximately 27 hours to compute for all nodes.

In [24]:
# Variable containing adjency dictionary of the directed graph
dic_adj = dict(Gdir.adjacency())
# Number of nodes of CO to compute the block ranking
n_C0_nodes = 50
# Name of CO category
C0 = 'English_footballers'

# Initiate block_ranking
block_ranking = [(0, C0)]
# Compute block ranking for all categories in categories_reduced
for cat in categories_reduced.keys():
    print(cat, end="; ") # Just to see which category is being proccessed
    if cat != C0:
        block_ranking.append((libgraph.distance(dic_adj, categories_reduced, C0, cat, n_C0_nodes), cat))
        
# Sort the block ranking by the score obtained
block_ranking.sort()
# Create pandas data frame to manipulate block ranking
pd_block_ranking = pd.DataFrame(block_ranking, columns=['Score','Category'])

English_footballers; The_Football_League_players; Association_football_forwards; Association_football_goalkeepers; Association_football_midfielders; Association_football_defenders; Living_people; Harvard_University_alumni; Major_League_Baseball_pitchers; Members_of_the_United_Kingdom_Parliament_for_English_constituencies; Indian_films; Year_of_death_missing; Year_of_birth_missing_(living_people); Rivers_of_Romania; Main_Belt_asteroids; Asteroids_named_for_people; English-language_albums; British_films; English-language_films; American_films; People_from_New_York_City; American_television_actors; American_film_actors; Debut_albums; Black-and-white_films; Year_of_birth_missing; Place_of_birth_missing_(living_people); American_military_personnel_of_World_War_II; Windows_games; 

In [25]:
# Display block ranking
display(pd_block_ranking)

,Score,Category
0,0.0,English_footballers
1,5.0,The_Football_League_players
2,6.0,American_film_actors
3,6.0,American_films
4,6.0,American_military_personnel_of_World_War_II
5,6.0,American_television_actors
6,6.0,Association_football_defenders
7,6.0,Association_football_forwards
8,6.0,Association_football_goalkeepers
9,6.0,Association_football_midfielders


## 2.2) Ranking nodes of each category 

In this section we are using block ranking computed before to rank the nodes of each category. We are following the 3 steps:

- [STEP1] Compute subgraph induced by C_0. For each node compute the sum of the weigths of the in-edges.
- [STEP2] Extend the graph to the nodes that belong to C_1. Thus, for each article in C_1 compute the score as before. Note that the in-edges coming from the previous category, C_0, have as weights the score of the node that sends the edge.
- [STEP3] Repeat Step2 up to the last category of the ranking. In the last step of the example you clearly see the weight update of the edge coming from node E.

We start adding attribute information of weight to all nodes to 0

In [26]:
#Initiate or restart weight of all nodes:
for node in set(Gdir.nodes):
    Gdir.node[node]['weight'] = 0

Now we are computing steps 1, 2 and 3

In [27]:
# Variable containing nodes with a computed weight
nodes_weight_computed = set()
# Loop arround all categories 
for index_cat in range(len(pd_block_ranking)):
    print(pd_block_ranking.loc[index_cat][1], end="; ") # Just to know which categorie is being computed
    CI = pd_block_ranking.loc[index_cat][1] # Category name
    
    #Compute subgraph of CI
    CI_sub = Gdir.subgraph(nodes=set(categories_reduced.get(CI))) # Make subgraph of the category

    #Compute node weights of CI
    for node in set(CI_sub.nodes):
        aux_weight = 0 # This variable is to sum at the end the weight of other nodes already computed
        # Loop in all in_edges nodes
        for node_adj in Gdir.in_edges(node):
            # Look if the in edges of the node to compute have already weight and save aux
            if node_adj in nodes_weight_computed:
                aux_weight += Gdir.node[node_adj[0]]['weight']
            
        # If node has not be computed, that means that belongs to the actual category
        if node not in nodes_weight_computed:
            # Save weight from subgraph and from nodes with computed weight
            Gdir.node[node]['weight'] = len(list(CI_sub.in_edges(node))) + aux_weight
    
    # Update set of computed nodes
    nodes_weight_computed.union(set(CI_sub.nodes))

English_footballers; The_Football_League_players; American_film_actors; American_films; American_military_personnel_of_World_War_II; American_television_actors; Association_football_defenders; Association_football_forwards; Association_football_goalkeepers; Association_football_midfielders; British_films; English-language_films; People_from_New_York_City; Black-and-white_films; Debut_albums; English-language_albums; Harvard_University_alumni; Indian_films; Living_people; Major_League_Baseball_pitchers; Members_of_the_United_Kingdom_Parliament_for_English_constituencies; Place_of_birth_missing_(living_people); Windows_games; Year_of_birth_missing; Year_of_birth_missing_(living_people); Year_of_death_missing; Asteroids_named_for_people; Rivers_of_Romania; Main_Belt_asteroids; 

### Node ranking results and analysis:

Making a list of list with the weight of the nodes of each category

In [28]:
# Make a score list with node_id and score of each category to print results
list_scores = []
for index_cat in range(len(pd_block_ranking)):
    print(pd_block_ranking.loc[index_cat][1], end="; ") # Just to know which categorie is being computed
    CI = pd_block_ranking.loc[index_cat][1] # Category name
    list_scores.append([])
    #Compute subgraph
    for node in set(categories_reduced.get(CI)):
        list_scores[index_cat].append((Gdir.node[node]['weight'], node))

English_footballers; The_Football_League_players; American_film_actors; American_films; American_military_personnel_of_World_War_II; American_television_actors; Association_football_defenders; Association_football_forwards; Association_football_goalkeepers; Association_football_midfielders; British_films; English-language_films; People_from_New_York_City; Black-and-white_films; Debut_albums; English-language_albums; Harvard_University_alumni; Indian_films; Living_people; Major_League_Baseball_pitchers; Members_of_the_United_Kingdom_Parliament_for_English_constituencies; Place_of_birth_missing_(living_people); Windows_games; Year_of_birth_missing; Year_of_birth_missing_(living_people); Year_of_death_missing; Asteroids_named_for_people; Rivers_of_Romania; Main_Belt_asteroids; 

Order nodes by weight of each list of categories:

In [29]:
# Order each list inside list_scores by score
for index_cat in range(len(pd_block_ranking)):
    list_scores[index_cat].sort(key = lambda x: x[0], reverse=True)

Print ordered nodes (weight, node_id) of each category:

In [30]:
# Print for each category a tuple of nodes with the score and the node_id
for index_cat in range(len(pd_block_ranking)):
    print(color.BOLD, index_cat, pd_block_ranking.loc[index_cat][1] + ':', color.END, *list_scores[index_cat][0:4])

 0 English_footballers:  (261, 82393) (163, 82365) (163, 81878) (133, 81847)
 1 The_Football_League_players:  (261, 82393) (195, 82092) (163, 81878) (156, 81856)
 2 American_film_actors:  (869, 1179311) (617, 1061960) (536, 1184448) (497, 1062053)
 3 American_films:  (212, 572855) (67, 1063315) (45, 1247389) (40, 1063898)
 4 American_military_personnel_of_World_War_II:  (809, 1400483) (193, 1400484) (181, 1025555) (177, 1169029)
 5 American_television_actors:  (536, 1184448) (401, 1184788) (387, 1184217) (361, 1061971)
 6 Association_football_defenders:  (105, 81871) (87, 88804) (79, 85428) (77, 81787)
 7 Association_football_forwards:  (210, 89923) (163, 82365) (144, 81928) (131, 83512)
 8 Association_football_goalkeepers:  (3125, 81952) (71, 81854) (67, 82589) (63, 82414)
 9 Association_football_midfielders:  (261, 82393) (245, 89978) (163, 81878) (156, 81856)
 10 British_films:  (1501, 1041937) (25, 1061229) (21, 1253706) (20, 1253703)
 11 English-language_films:  (67, 1063315) (64,

Now we are printing the same but changing the node_id for the real name of the article

In [31]:
for index_cat in range(len(pd_block_ranking)):
    names = []
    for score, id_ in list_scores[index_cat]:
        names.append(names_list[id_])
        
    print(color.BOLD + str(index_cat) + '. ' + pd_block_ranking.loc[index_cat][1] + color.END, *names[:5], sep="; ")

0. English_footballers; David Beckham; Wayne Rooney; Harry Redknapp; Sam Allardyce; Glenn Hoddle
1. The_Football_League_players; David Beckham; Kevin Keegan; Harry Redknapp; Roy Keane; Kenny Dalglish
2. American_film_actors; Madonna (entertainer); Steven Spielberg; Britney Spears; Arnold Schwarzenegger; Whitney Houston
3. American_films; Our Gang; The Wizard of Oz (1939 film); The Three Mesquiteers; Halloween (1978 film); Raiders of the Lost Ark
4. American_military_personnel_of_World_War_II; Jimmy Carter; Richard Nixon; Tony Bennett; Henry Kissinger; Lyndon B. Johnson
5. American_television_actors; Britney Spears; Christina Aguilera; Jennifer Lopez; Clint Eastwood; Cher
6. Association_football_defenders; Micky Adams; Giovanni Trapattoni; Luiz Felipe Scolari; Mick McCarthy; Glenn Roeder
7. Association_football_forwards; Pel; Wayne Rooney; Kenny Dalglish; Thierry Henry; Alan Shearer
8. Association_football_goalkeepers; Goalkeeper (association football); Peter Schmeichel; Peter Shilton; 

As we can see all results we have a list of categories block ranked and also each article ranked by number of in-nodes.